In [13]:
import warnings
import numpy as np
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

In [2]:
%config Completer.use_jedi = False  # 자동완성 속도 증가
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
class NaiveDense:
    def __init__(self, input_size, output_size, activation):
        self.activation = activation
        
        w_shape = (input_size, output_size) # 랜덤한 값으로 초기화된 (input_size, output_size)크기의 행렬 W를 만듬
        w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
        self.W = tf.Variable(w_initial_value)
        
        b_shape = (output_size)
        b_initial_value = tf.zeros(b_shape)
        self.b = tf.Variable(b_initial_value)
    
    def __call__(self, inputs):
        return self.activation(tf.matmul(inputs, self.W) + self.b)
    
    @property
    def weights(self):
        return [self.W, self.b]

In [4]:
class NaiveSequential:
    def __init__(self, layers):
        self.layers = layers
        
    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x)
        return x
    
    @property
    def weights(self):
        weights = []
        for layer in self.layers:
            weights += layer.weights
        return weights

In [5]:
model = NaiveSequential([
    NaiveDense(input_size=28 * 28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)
])
assert len(model.weights) == 4

In [6]:
import math

class BatchGenerator:
    def __init__(self, images, labels, batch_size=128):
        assert len(images) == len(labels)
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = math.ceil(len(images) / batch_size)
        
    def next(self):
        images = self.images[self.index : self.index + self.batch_size]
        labels = self.labels[self.index : self.index + self.batch_size]
        self.index += self.batch_size
        return images, labels

In [7]:
def one_training_step(model, images_batch, labels_batch):
    with tf.GradientTape() as tape:
        predictions = model(images_batch)
        per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(
            labels_batch, predictions)
        average_loss = tf.reduce_mean(per_sample_losses)
    gradients = tape.gradient(average_loss, model.weights)
    update_weights(gradients, model.weights)
    return average_loss

In [8]:
learning_rate = 1e-3

def update_weights(gradients, weights):
    for g, w in zip(gradients, weights):
        w.assign_sub(g * learning_rate)

In [9]:
from tensorflow.keras import optimizers

optimizer = optimizers.SGD(learning_rate=1e-3)

def update_weights(gradients, weights):
    optimizer.apply_gradients(zip(gradients, weights))

In [10]:
def fit(model, images, labels, epochs, batch_size=128):
    for epoch_counter in range(epochs):
        print(f'에포크 {epoch_counter}')
        batch_generator = BatchGenerator(images, labels)
        for batch_counter in range(batch_generator.num_batches):
            images_batch, labels_batch = batch_generator.next()
            loss = one_training_step(model, images_batch, labels_batch)
            if batch_counter % 100 == 0:
                print(f'{batch_counter}번째 배치 손실: {loss:.2f}')

In [11]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

fit(model, train_images, train_labels, epochs=10, batch_size=128)

에포크 0
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
0번째 배치 손실: 7.18
100번째 배치 손실: 2.23
200번째 배치 손실: 2.24
300번째 배치 손실: 2.10
400번째 배치 손실: 2.21
에포크 1
0번째 배치 손실: 1.92
100번째 배치 손실: 1.88
200번째 배치 손실: 1.87
300번째 배치 손실: 1.72
400번째 배치 손실: 1.83
에포크 2
0번째 배치 손실: 1.60
100번째 배치 손실: 1.58
200번째 배치 손실: 1.54
300번째 배치 손실: 1.44
400번째 배치 손실: 1.51
에포크 3
0번째 배치 손실: 1.34
100번째 배치 손실: 1.34
200번째 배치 손실: 1.27
300번째 배치 손실: 1.22
400번째 배치 손실: 1.28
에포크 4
0번째 배치 손실: 1.14
100번째 배치 손실: 1.16
200번째 배치 손실: 1.07
300번째 배치 손실: 1.05
400번째 배치 손실: 1.11
에포크 5
0번째 배치 손실: 1.00
100번째 배치 손실: 1.02
200번째 배치 손실: 0.92
300번째 배치 손실: 0.93
400번째 배치 손실: 0.99
에포크 6
0번째 배치 손실: 0.88
100번째 배치 손실: 0.91
200번째 배치 손실: 0.81
300번째 배치 손실: 0.84
400번째 배치 손실: 0.90
에포크 7
0번째 배치 손실: 0.80
100번째 배치 손실: 0.82
200번째 배치 손실: 0.73
300번째 배치 손실: 0.77
400번째 배치 손실: 0.83
에포크 8
0번째 배치 손실: 0.73
100번째 배치 손실: 0.76
200번째 배치 손실: 0.66
300번째 배치 손실: 0.71
400번째 배치 손실: 0.78
에포크 9
0번째 배치 손실: 0.68
100번째 배치 손실: 0.70
200번째 배치 손실: 0.61
30

In [12]:
predictions = model(test_images)
predictions = predictions.numpy()
predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels
print(f'정확도: {matches.mean():.2f}')

정확도: 0.81
